In [ ]:
!nvidia-smi

In [ ]:
# Download Ollama
!curl https://ollama.ai/install.sh | sh
# Install required libraries
!pip install aiohttp pyngrok

import os
import asyncio

# Update LD_LIBRARY_PATH to include NVIDIA library directories
os.environ.update({
    'LD_LIBRARY_PATH': '/usr/local/nvidia/lib64:/usr/local/cuda-12.3/lib64:/usr/local/cuda-12.3/compat/libcuda.so'
})

async def run_process(cmd):
    print('>>> starting', *cmd)
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

# Register an account at ngrok.com and create an authtoken, and place it here
await asyncio.gather(
    run_process(['ngrok', 'config', 'add-authtoken', 'your-authtoken'])
)


In [ ]:
# Start ollama and ngrok
await asyncio.gather(
    run_process(['ollama', 'serve']),
    run_process(['ngrok', 'http', '--log', 'stderr', '11434', '--host-header', 'localhost:11434'])
)

### Model uploaing 👇

In [ ]:
# Step 1: List the contents of the Ollama models directory
import os
print("Listing the contents of /root/.ollama/models/")
print(os.listdir("/root/.ollama/models/"))

# Step 2: Create the target directory on Kaggle's working environment
print("Creating the directory /kaggle/working/ollama_models/")
os.makedirs("/kaggle/working/ollama_models", exist_ok=True)

# Step 3: Copy the contents from the Ollama model directory to the target directory
import shutil
print("Copying models to /kaggle/working/ollama_models/")
shutil.copytree("/root/.ollama/models", "/kaggle/working/ollama_models", dirs_exist_ok=True)

# Step 4: Install Kaggle Hub
print("Installing kagglehub package")
!pip install kagglehub

# Step 5: Set Kaggle username and key for authentication
import os
os.environ["KAGGLE_USERNAME"] = ""  # Replace with your actual Kaggle username
os.environ["KAGGLE_KEY"] = ""  # Replace with your actual Kaggle API key

# Step 6: Upload model to Kaggle Hub
import kagglehub

# Set your model handle and local model directory path
handle = "KAGGLE_USERNAME/model-name/package/version"  # Replace with your model handle
local_model_dir = "/kaggle/working/ollama_models/"

# Upload the model with optional version notes
kagglehub.model_upload(handle, local_model_dir, version_notes="Upload date :- 02/12/2024")

# Optional: You can also specify a license
# kagglehub.model_upload(handle, local_model_dir, license_name="Apache 2.0")

# Optional: Specify patterns to ignore during upload
# kagglehub.model_upload(handle, local_model_dir, ignore_patterns=["original/", "*.tmp"])

print("Model upload complete!")


## Soft-link model 👇

In [ ]:
import os
import shutil

def sync_or_copy(source_dir, target_dir, folder_name):
    """
    Handle syncing or copying a folder based on its existence in the target directory.
    
    If the filesystem is read-only, it will skip the write operation.
    
    Parameters:
    - source_dir (str): Path to the source directory containing the folder to copy.
    - target_dir (str): Path to the target directory where the folder should be copied or synced.
    - folder_name (str): Name of the folder to process (e.g., 'manifests', 'blobs').
    """
    source_folder = os.path.join(source_dir, folder_name)
    target_folder = os.path.join(target_dir, folder_name)

    if os.path.exists(target_folder):
        print(f"'{folder_name}' folder exists at {target_folder}. Copying contents...")
        for item in os.listdir(source_folder):
            src_item = os.path.join(source_folder, item)
            dest_item = os.path.join(target_folder, item)
            
            if os.path.isdir(src_item):
                if os.path.exists(dest_item):
                    print(f"Directory exists, merging: {dest_item}")
                try:
                    shutil.copytree(src_item, dest_item, dirs_exist_ok=True)
                except OSError as e:
                    print(f"Skipping {src_item}: {e}")
            else:
                try:
                    shutil.copy2(src_item, dest_item)
                except OSError as e:
                    print(f"Skipping {src_item}: {e}")
    else:
        print(f"'{folder_name}' folder does not exist at {target_dir}. Copying the entire folder...")
        try:
            shutil.copytree(source_folder, target_folder)
        except OSError as e:
            print(f"Error copying {folder_name}: {e}")

# Define source and target paths
source_dir = "/kaggle/input/lama3_2_1b/pytorch/v1/3"
target_dir = "/root/.ollama/models"

# Process the manifests and blobs folders
sync_or_copy(source_dir, target_dir, "manifests")
sync_or_copy(source_dir, target_dir, "blobs")
